# Data Exploration

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import duckdb
import pickle

In [5]:
# Display Options for Pandas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = None

In [3]:
data_path = Path("../data")
data_path_string = data_path.resolve().as_posix()

### ETL and EDA

In [4]:
con = duckdb.connect(database=f"{data_path}/spotify.db")

In [9]:
def clean_db(con: duckdb.DuckDBPyConnection):
    con.execute("drop table if exists lyrics_features")
    con.execute("drop table if exists low_level_audio_features")
    con.execute("drop table if exists albums")
    con.execute("drop table if exists artists")
    con.execute("drop table if exists tracks")
    con.execute("drop table if exists features")
    con.execute("drop view if exists lookup")

clean_db(con)

In [11]:
def load_data_into_db(con: duckdb.DuckDBPyConnection, data_path_string: str):
    con.read_csv(f"{data_path_string}/lyrics_features.csv", all_varchar=False).create("lyric_features")
    con.read_csv(f"{data_path_string}/low_level_audio_features.csv", all_varchar=False).create("low_level_audio_features")
    con.read_csv(f"{data_path_string}/spotify_albums.csv", all_varchar=False).create("albums")
    con.read_csv(f"{data_path_string}/spotify_artists.csv", all_varchar=False).create("artists")
    con.read_csv(f"{data_path_string}/spotify_tracks.csv", all_varchar=False).create("tracks")

load_data_into_db(con, data_path_string)

In [13]:
con.table("lyric_features").limit(3).df()

column0  mean_syllables_word  mean_words_sentence  n_sentences  n_words  \
0        0                -1.00                -1.00           -1       -1   
1        1                 1.10                 5.65           31      326   
2        2                 1.37                 4.77           74      532   

   sentence_similarity                track_id  vocabulary_wealth  
0            -1.000000  5KIfHjHI5NIsPHNt58qua0              -1.00  
1             0.043011  13keyz9ikBe6ZpRasw7l4X               0.45  
2             0.050352  1WugzepXsLjnsM0K4UaWYc               0.59

In [14]:
con.table("low_level_audio_features").limit(3).df()


column000  Chroma_1  Chroma_10  Chroma_11  Chroma_12  Chroma_2  Chroma_3  \
0          0  0.438296   0.472769   0.427441   0.436688  0.467697  0.493862   
1          1  0.596605   0.368288   0.285263   0.302211  0.905805  0.510909   
2          2  0.505224   0.500420   0.506773   0.488258  0.498356  0.573582   

   Chroma_4  Chroma_5  Chroma_6  Chroma_7  Chroma_8  Chroma_9       MEL_1  \
0  0.512244  0.568658  0.560524  0.513068  0.544648  0.544643    0.000140   
1  0.221708  0.311248  0.491277  0.416469  0.411171  0.553497    0.053749   
2  0.690761  0.742858  0.686282  0.657118  0.550437  0.491275  381.736538   

      MEL_10   MEL_100   MEL_101   MEL_102   MEL_103   MEL_104   MEL_105  \
0   0.667637  0.000016  0.000009  0.000006  0.000005  0.000005  0.000005   
1  20.364112  0.022766  0.019990  0.013676  0.015588  0.016917  0.022716   
2  50.096990  0.166018  0.159811  0.180781  0.169495  0.153824  0.150577   

    MEL_106   MEL_107   MEL_108   MEL_109     MEL_11   MEL_110   MEL_111  \
0  0.000005  0.000005  0.000005  0.000005   1.036885  0.000005  0.000005   
1  0.020967  0.021773  0.015938  0.014059  24.822703  0.008655  0.005350   
2  0.122907  0.104753  0.088342  0.074665  41.289397  0.060141  0.040727   

    MEL_112   MEL_113   MEL_114   MEL_115   MEL_116   MEL_117       MEL_118  \
0  0.000005  0.000005  0.000005  0.000006  0.000006  0.000004  2.706399e-07   
1  0.003345  0.002055  0.001265  0.001166  0.001026  0.000534  1.876189e-05   
2  0.030447  0.018008  0.012491  0.009160  0.006501  0.002696  1.114347e-04   

        MEL_119     MEL_12       MEL_120       MEL_121       MEL_122  \
0  3.611252e-09   0.960690  3.454723e-09  3.324464e-09  3.249713e-09   
1  5.515725e-08  19.904134  4.847649e-08  4.373043e-08  3.991839e-08   
2  6.437100e-06  25.838512  6.474858e-06  5.948196e-06  5.798902e-06   

        MEL_123       MEL_124       MEL_125       MEL_126       MEL_127  \
0  3.211787e-09  3.196705e-09  3.123937e-09  3.096601e-09  3.132088e-09   
1  3.699672e-08  3.468608e-08  3.284620e-08  3.159500e-08  3.051798e-08   
2  6.219467e-06  6.090780e-06  5.806435e-06  6.175595e-06  6.012861e-06   

        MEL_128     MEL_13     MEL_14     MEL_15      MEL_16     MEL_17  \
0  3.115733e-09   1.309046   1.727061   1.172454    2.451283   2.383262   
1  2.988750e-08  29.142250  27.009086  13.279951   11.811473  34.071757   
2  5.571497e-06  26.049473  49.293065  76.258536  115.575303  35.116960   

      MEL_18     MEL_19       MEL_2     MEL_20     MEL_21     MEL_22  \
0   3.495376   1.708026    0.001025   1.812995   2.062649   1.269325   
1  73.849335  16.803801    1.346285  10.113495   5.480764  17.990936   
2  29.367355  23.180991  539.954759  49.485727  48.526434  24.819987   

      MEL_23     MEL_24     MEL_25     MEL_26     MEL_27     MEL_28    MEL_29  \
0   0.964405   0.662044   0.739214   0.527574   0.688505   0.454942  0.311517   
1  19.527366   5.496805  13.353018  41.659928  81.923985  22.629720  3.932411   
2  26.903197  44.271672  25.120495  14.562212  15.190511   8.085279  7.609208   

        MEL_3    MEL_30    MEL_31    MEL_32    MEL_33    MEL_34     MEL_35  \
0    0.017772  0.296182  0.268481  0.286426  0.314140  0.346320   0.244440   
1    7.441805  2.977619  2.733937  2.493947  1.930312  6.046661  16.889717   
2  368.558497  8.200001  7.657170  7.623656  3.147984  3.046287   2.332731   

      MEL_36    MEL_37    MEL_38     MEL_39       MEL_4    MEL_40    MEL_41  \
0   0.223053  0.363009  0.532202   0.552576    0.292901  0.418687  0.308175   
1  24.328768  9.966199  4.330980  10.608160   34.019282  6.376572  2.894632   
2   2.379385  1.651035  1.931251   2.763406  309.964223  1.823783  1.508735   

     MEL_42    MEL_43    MEL_44    MEL_45    MEL_46    MEL_47     MEL_48  \
0  0.248862  0.204990  0.144929  0.106920  0.073315  0.041151   0.037855   
1  3.903454  4.606321  1.174320  0.634846  0.725404  4.232507  16.313335   
2  1.729277  3.145477  4.454020  2.488309  1.283118  1.844057   2.482388   

     MEL_4

In [16]:
con.table("albums").limit(1).df()

column00 album_type               artist_id  \
0         0     single  3DiDSECUqqY1AuBP8qtaIa   

                                                                                     available_markets  \
0  ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...   

                                                          external_urls  \
0  {'spotify': 'https://open.spotify.com/album/1gAM7M4rBwEbSPeAQR2nx1'}   

                                                       href  \
0  https://api.spotify.com/v1/albums/1gAM7M4rBwEbSPeAQR2nx1   

                       id  \
0  1gAM7M4rBwEbSPeAQR2nx1   

                                                                                                images  \
0  [{'height': 640, 'url': 'https://i.scdn.co/image/5872e4d8fac4ef7552576d481b1d676189b4056a', 'wid...   

                    name release_date release_date_precision  total_tracks  \
0  If I Ain't Got You EP   2019-02-08                    day             6   

                 track_id track_name_prev  \
0  2iejTMy9XZ8Gaae0aQ2yl0        track_32   

                                    uri   type  
0  spotify:album:1gAM7M4rBwEbSPeAQR2nx1  album

In [17]:
con.table("artists").limit(3).df()

column0  artist_popularity  followers  \
0        0                 44      23230   
1        1                 22        313   
2        2                 26       1596   

                                                                               genres  \
0  ['sertanejo', 'sertanejo pop', 'sertanejo tradicional', 'sertanejo universitario']   
1                                                                                  []   
2                                                                 ['danish pop rock']   

                       id            name                track_id  \
0  4mGnpjhqgx4RUdsIJiURdo   Juliano Cezar  0wmDmAILuW9e2aRttkl4aC   
1  1dLnVku4VQUOLswwDFvRc9  The Grenadines  4wqwj0gA8qPZKLl5WVqXml   
2  6YVY310fjfUzKi8hiqR7iK         Gangway  1bFqWDbvHmZe2f4Nf9qaD8   

  track_name_prev    type  
0         track_9  artist  
1        track_30  artist  
2        track_38  artist

In [18]:
con.table("tracks").limit(3).df()

column00  acousticness                album_id  \
0         0         0.294  0D3QufeCudpQANOR7luqdr   
1         1         0.863  1bcqsH5UyTBzmh9YizdsBE   
2         2         0.750  4tKijjmxGClg4JOLAyo2qE   

                                                       analysis_url  \
0  https://api.spotify.com/v1/audio-analysis/5qljLQuKnNJf4F4vfxQB0V   
1  https://api.spotify.com/v1/audio-analysis/3VAX2MJdmdqARLSU5hPMpm   
2  https://api.spotify.com/v1/audio-analysis/1L3YAhsEMrGVvCgDXj2TYn   

                   artists_id  \
0  ['3mxJuHRn2ZWD5OofvJtDZY']   
1  ['4xWMewm6CYMstu0sPgd9jJ']   
2  ['3hYaK5FF3YAglCj5HZgBnP']   

                                                                                     available_markets  \
0  ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...   
1  ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...   
2                                                                                               ['GB']   

  country  danceability  disc_number  duration_ms  energy  \
0      BE         0.698          1.0     235584.0   0.606   
1      BE         0.719          1.0     656960.0   0.308   
2      BE         0.466          1.0     492840.0   0.931   

                                                       href  \
0  https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V   
1  https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm   
2  https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn   

                       id  instrumentalness   key  liveness  loudness  \
0  5qljLQuKnNJf4F4vfxQB0V          0.000003  10.0     0.151    -7.447   
1  3VAX2MJdmdqARLSU5hPMpm          0.000000   6.0     0.253   -10.340   
2  1L3YAhsEMrGVvCgDXj2TYn          0.000000   4.0     0.938   -13.605   

                                                                                                lyrics  \
0  \r\n\r\nPerhaps I am bound to be restless\r\nAlways yearning, never satisfied\r\nPerhaps I'm ach...   
1  \r\nYour Gods and my Gods-do you or I know which are the stronger? Native Proverb.\r\nEAST of Su...   
2  \r\n\r\nClosed off from love, I didn't need the pain\r\nOnce or twice was enough and it was all ...   

   mode                                             name  \
0   0.0                                            Blood   
1   1.0                                The Ugly Duckling   
2   0.0  Jimmy Launches His Own Range Of Greetings Cards   

                 playlist  popularity  \
0         Hipsteribrunssi        28.0   
1          Animal Stories        31.0   
2  Best Of British Comedy        31.0   

                                                                                           preview_url  \
0  https://p.scdn.co/mp3-preview/1b05a902da3a251d07a38aa710ffae559fc33d08?cid=b3cdb16d0df2409abf6a8...   
1  https://p.scdn.co/mp3-preview/d8140736a6131cb5595f061975173a272c343a0a?cid=b3cdb16d0df2409abf6a8...   
2  https://p.scdn.co/mp3-preview/c8af28fb15185b18977152eb50eefef8d90af5a2?cid=b3cdb16d0df2409abf6a8...   

   speechiness    tempo  time_signature  \
0       0.0262  115.018             4.0   
1       0.9220  115.075             3.0   
2       0.9440   79.565             4.0   

                                                 track_href track_name_prev  \
0  https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V        track_14   
1  https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm         track_3   
2  https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn         track_4   

   track_number                                   uri  valence   type  
0           1.0  spotify:track:5qljLQuKnNJf4F4vfxQB0V    0.622  track  
1           3.0  spotify:track:3VAX2MJdmdqARLSU5hPMpm    0.589  track  
2           4.0  spotify:track:1L3YAhsEMrGVvCgDXj2TYn    0.085  track

In [25]:
# Construct Lookup View
def create_lookup_table(con: duckdb.DuckDBPyConnection):
    sql = """
        drop view if exists lookup;
        create view lookup as
        select 
            t.id, t.name as track_name, ar.name as artist_name, a.name as album_name, 
            t.preview_url, t.track_href, t.analysis_url
        from tracks t
        left join albums a on t.album_id = a.id
        left join artists ar on a.artist_id = ar.id
        """
    con.execute(sql)

create_lookup_table(con)

In [20]:
con.view("lookup").limit(3).df()

id         track_name            artist_name  \
0  5qljLQuKnNJf4F4vfxQB0V              Blood           Jesse Markin   
1  3VAX2MJdmdqARLSU5hPMpm  The Ugly Duckling  Favorite Kids Stories   
2  4PrAZpH9Ic7S47E78BN6E4       Already Gone                Alvvays   

                   album_name  \
0                       Blood   
1  Storytime Classics, Vol. 2   
2              Antisocialites   

                                                                                           preview_url  \
0  https://p.scdn.co/mp3-preview/1b05a902da3a251d07a38aa710ffae559fc33d08?cid=b3cdb16d0df2409abf6a8...   
1  https://p.scdn.co/mp3-preview/d8140736a6131cb5595f061975173a272c343a0a?cid=b3cdb16d0df2409abf6a8...   
2  https://p.scdn.co/mp3-preview/10a9607c178fe1eaeb3d12aed9cee82da4963f24?cid=b3cdb16d0df2409abf6a8...   

                                                 track_href  \
0  https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V   
1  https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm   
2  https://api.spotify.com/v1/tracks/4PrAZpH9Ic7S47E78BN6E4   

                                                       analysis_url  
0  https://api.spotify.com/v1/audio-analysis/5qljLQuKnNJf4F4vfxQB0V  
1  https://api.spotify.com/v1/audio-analysis/3VAX2MJdmdqARLSU5hPMpm  
2  https://api.spotify.com/v1/audio-analysis/4PrAZpH9Ic7S47E78BN6E4

In [26]:
# Construct Features Table
def create_features_table(con: duckdb.DuckDBPyConnection):
    sql = """
        drop table if exists features;
        create table features as
        select
            t.id, t.acousticness, t.danceability, t.energy, t.instrumentalness, 
            t.liveness, t.loudness, t.speechiness, t.tempo, t.valence
        from tracks t
        """
    con.execute(sql)

create_features_table(con)

In [23]:
con.query("summarize features").df()

column_name column_type                     min  \
0                id     VARCHAR  000RDCYioLteXcutOjeweY   
1      acousticness      DOUBLE                     0.0   
2      danceability      DOUBLE                     0.0   
3            energy      DOUBLE                     0.0   
4  instrumentalness      DOUBLE                     0.0   
5          liveness      DOUBLE                     0.0   
6          loudness      DOUBLE                   -60.0   
7       speechiness      DOUBLE                     0.0   
8             tempo      DOUBLE                     0.0   
9           valence      DOUBLE                     0.0   

                      max approx_unique                  avg  \
0  7zzpno7uAqkAzWZDQuGEFA        101691                  NaN   
1                   0.996          4628   0.3521235614982393   
2                   0.989          1199   0.5860150295765171   
3                     1.0          2353   0.5864791632319346   
4                     1.0          5341  0.14877590260086768   
5                   0.999          1715  0.19764030645778108   
6                   2.719         22382   -9.462719675492316   
7                   0.969          1638   0.1288414963850934   
8                 244.035         50300   118.35852701125256   
9                   0.993          1762   0.4828126257801232   

                   std                   q25                     q50  \
0                  NaN                   NaN                     NaN   
1  0.33485523897742264  0.040886282525419884      0.2379812289449274   
2  0.17772425685883975    0.4797636257135789      0.6102385939777873   
3   0.2601704025337212    0.4111254863222407      0.6284573068498541   
4   0.3040235636638487                   0.0  3.8518588797877756e-05   
5  0.17539085924060754   0.09555992164363636     0.12374811787729091   
6    6.198508047679088   -11.157471170426556       -7.60183592122598   
7   0.2033237132844151   0.03637234719882735    0.050731470284077214   
8    30.22407426199396     95.87399140685004       118.2468924909389   
9  0.26169048447594423   0.27115585393477853      0.4769632390749201   

                   q75   count null_percentage  
0                  NaN  101939            0.0%  
1   0.6444174472884937  101939            0.0%  
2   0.7144499849384677  101939            0.0%  
3   0.7977735646075533  101939            0.0%  
4  0.03539115443795795  101939            0.0%  
5  0.24154309151390566  101939            0.0%  
6   -5.512450857917222  101939            0.0%  
7  0.10407614620018696  101939            0.0%  
8   136.20637236145168  101939            0.0%  
9   0.6933812005186372  101939            0.0%

## ML

In [5]:
def load_data_into_pd(con: duckdb.DuckDBPyConnection) -> pd.DataFrame:
    return con.query("select * from features").df()

df = load_data_into_pd(con)

In [28]:
df.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence'],
      dtype='object')

In [29]:
df.describe()

acousticness   danceability         energy  instrumentalness  \
count  101939.000000  101939.000000  101939.000000     101939.000000   
mean        0.352124       0.586015       0.586479          0.148776   
std         0.334855       0.177724       0.260170          0.304024   
min         0.000000       0.000000       0.000000          0.000000   
25%         0.040700       0.480000       0.411000          0.000000   
50%         0.238000       0.610000       0.629000          0.000037   
75%         0.645000       0.714000       0.798000          0.034400   
max         0.996000       0.989000       1.000000          1.000000   

            liveness       loudness    speechiness          tempo  \
count  101939.000000  101939.000000  101939.000000  101939.000000   
mean        0.197640      -9.462720       0.128841     118.358527   
std         0.175391       6.198508       0.203324      30.224074   
min         0.000000     -60.000000       0.000000       0.000000   
25%         0.095600     -11.149000       0.036400      95.973000   
50%         0.124000      -7.599000       0.050600     118.067000   
75%         0.241000      -5.509000       0.104000     136.045000   
max         0.999000       2.719000       0.969000     244.035000   

             valence  
count  101939.000000  
mean        0.482813  
std         0.261690  
min         0.000000  
25%         0.271000  
50%         0.477000  
75%         0.693000  
max         0.993000

In [6]:
if Path(data_path / "pipeline.pkl").exists():
    print("hello")

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

def get_pipeline(df) -> Pipeline:
    # Check if pipeline exists
    if Path(data_path / "pipeline.pkl").exists():
        with open(data_path / "pipeline.pkl", "rb") as f:
            return pickle.load(f)
    
    # Otherwise, create pipeline
    pipeline = Pipeline([ ("scaler", MinMaxScaler()) ])
    pipeline.fit(df.loc[:, df.columns != "id"])
    with open(data_path / "pipeline.pkl", "wb") as f:
        pickle.dump(pipeline, f)
        
    return pipeline

pipe = get_pipeline(df)

In [13]:
def preprocess_data(df: pd.DataFrame) -> np.ndarray:
    pipe = get_pipeline(df)
    X = pipe.transform(df.loc[:, df.columns != "id"])
    return X

X = preprocess_data(df)

In [27]:
from sklearn.neighbors import NearestNeighbors

def get_model_knn(X: np.ndarray or None = None) -> NearestNeighbors:
    # Check if model exists
    if Path(data_path / "knn.pkl").exists():
        with open(data_path / "knn.pkl", "rb") as f:
            return pickle.load(f)

    # Otherwise, create model
    knn = NearestNeighbors(n_neighbors=5, algorithm="ball_tree")
    knn.fit(X)
    with open(data_path / "knn.pkl", "wb") as f:
        pickle.dump(knn, f)

    return knn

knn = get_model_knn(X)

In [12]:
# lookup ids
def lookup_ids(con: duckdb.DuckDBPyConnection, lookup_query: str) -> str:
    sql = """
      select *
      from lookup
      where 
        regexp_matches(lower(concat(track_name, ' ', artist_name)), $param)
      limit 10
    """
    return con.execute(sql, { "param": lookup_query.lower()}).fetch_df()

df_songs = lookup_ids(con, "like a prayer")
df_songs

id     track_name artist_name  \
0  0OuGlX0EnsXQ4vvOunF9A3  Like a Prayer  Goshfather   
1  2v7ywbUzCgcVohHaKUcacV  Like a Prayer     Madonna   
2  1z3ugFmUKoCzGsI6jdY4Ci  Like a Prayer     Madonna   

                          album_name  \
0                      Like a Prayer   
1                      Like a Prayer   
2  Celebration (double disc version)   

                                                                                           preview_url  \
0  https://p.scdn.co/mp3-preview/dbe9844b5b38df522bff03e411aedb6c52c87559?cid=b3cdb16d0df2409abf6a8...   
1  https://p.scdn.co/mp3-preview/274e7167b6222fc3f8e167bf036c7286bf265f2d?cid=b3cdb16d0df2409abf6a8...   
2  https://p.scdn.co/mp3-preview/b56a70770267b00ccae13c2e8c8a34ed54627d02?cid=b3cdb16d0df2409abf6a8...   

                                                 track_href  \
0  https://api.spotify.com/v1/tracks/0OuGlX0EnsXQ4vvOunF9A3   
1  https://api.spotify.com/v1/tracks/2v7ywbUzCgcVohHaKUcacV   
2  https://api.spotify.com/v1/tracks/1z3ugFmUKoCzGsI6jdY4Ci   

                                                       analysis_url  
0  https://api.spotify.com/v1/audio-analysis/0OuGlX0EnsXQ4vvOunF9A3  
1  https://api.spotify.com/v1/audio-analysis/2v7ywbUzCgcVohHaKUcacV  
2  https://api.spotify.com/v1/audio-analysis/1z3ugFmUKoCzGsI6jdY4Ci

In [13]:
# Select Song
df_test = df.query("id == '2v7ywbUzCgcVohHaKUcacV'")
df_test

id  acousticness  danceability  energy  \
85789  2v7ywbUzCgcVohHaKUcacV         0.432          0.66   0.629   

       instrumentalness  liveness  loudness  speechiness    tempo  valence  
85789          0.000073     0.172   -12.359       0.0387  111.926    0.324

In [21]:
def make_recommendations(con: duckdb.DuckDBPyConnection, df: pd.DataFrame, id: str, n_neighbors: int = 10) -> pd.DataFrame:
    # Predict
    df_test = df.query(f"id == '{id}'")
    X_test = preprocess_data(df_test)
    knn = get_model_knn()
    distances, indices = knn.kneighbors(X_test, n_neighbors=n_neighbors)
    ids = df.loc[indices[0].tolist(), :]

    # Lookup song details
    sql = """
        select * 
        from lookup
        where id in (select id from ids)
    """
    details = con.query(sql).df()

    # Merge
    merge = ids.merge(details, how="left", on="id")
    return merge


In [22]:
make_recommendations(con, df, id="2v7ywbUzCgcVohHaKUcacV")


id  acousticness  danceability  energy  \
0  2v7ywbUzCgcVohHaKUcacV         0.432         0.660   0.629   
1  7mrsLJSTzTMRQjEzWDAlOw         0.421         0.619   0.653   
2  0flFPv83zCKHwzuhlEDILy         0.416         0.633   0.682   
3  5axqYa8jADY4TKO9jMr7Zt         0.503         0.607   0.664   
4  0fqjy5gVNVmQQmxOLLfb3c         0.400         0.638   0.687   
5  68vdi4VhdQ3JTTRWPi5z6U         0.447         0.629   0.645   
6  4ZmAMOU0bcmrwwOvEK8aDT         0.407         0.626   0.574   
7  3mY5W2fJF9wKoy5d4IOtsZ         0.445         0.651   0.594   
8  0wl2rHiiQRHAW41UHYOI1u         0.414         0.660   0.599   
9  1lIz3Wn8b1lBsl0pvOpmU7         0.491         0.599   0.661   

   instrumentalness  liveness  loudness  speechiness    tempo  valence  \
0          0.000073    0.1720   -12.359       0.0387  111.926    0.324   
1          0.000080    0.1470    -8.791       0.0294  118.958    0.327   
2          0.000105    0.1480    -6.547       0.0359  114.999    0.324   
3          0.002130    0.1150   -10.376       0.0314  110.010    0.330   
4          0.000114    0.1660    -6.556       0.0351  114.980    0.323   
5          0.000074    0.0992   -10.735       0.1030  115.077    0.283   
6          0.000000    0.1700    -7.730       0.0300  119.531    0.272   
7          0.000000    0.1230    -5.879       0.0404  113.971    0.300   
8          0.012400    0.1270    -9.132       0.0271  135.066    0.336   
9          0.000002    0.1530    -8.140       0.0529  124.052    0.306   

                                    track_name      artist_name  \
0                                Like a Prayer          Madonna   
1                         Painted In The Sound   Justus Proffit   
2                               Take Your Time        Vance Joy   
3                                        Lloré  Monsieur Periné   
4                               Take Your Time        Vance Joy   
5  Electric (feat. Khalid) [Ryan Riback Remix]      Alina Baraz   
6                                  Quién Diría   Ricardo Arjona   
7         Anywhere With You (feat. Andie Nora)          Wallaby   
8                                 Family Tapes      Dead Horses   
9                                 Watching You          Lea Rue   

                             album_name  \
0                         Like a Prayer   
1                    L.A.'s Got Me Down   
2                         Nation of Two   
3     Caja De Música (Edición Especial)   
4                         Nation Of Two   
5            The Color Of You (Remixes)   
6                     Canciones De Amor   
7  Anywhere With You (feat. Andie Nora)   
8                          Family Tapes   
9                          Watching You   

                                         preview_url  \
0  https://p.scdn.co/mp3-preview/274e7167b6222fc3...   
1  https://p.scdn.co/mp3-preview/d5277b8f48b35e00...   
2  https://p.scdn.co/mp3-preview/6841b4be0f54da98...   
3  https://p.scdn.co/mp3-preview/fc993da8974873c4...   
4  https://p.scdn.co/mp3-preview/c4defe3d34b142df...   
5  https://p.scdn.co/mp3-preview/506689b5b743e367...   
6  https://p.scdn.co/mp3-preview/319ca1a607c2ce81...   
7  https://p.scdn.co/mp3-preview/8da4a78de402994d...   
8  https://p.scdn.co/mp3-preview/9c1dbca4aac171c1...   
9  https://p.scdn.co/mp3-preview/cda37d5344b7e0b3...   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/2v7ywbUzCgcV...   
1  https://api.spotify.com/v1/tracks/7mrsLJSTzTMR...   
2  https://api.spotify.com/v1/tracks/0flFPv83zCKH...   
3  https://api.spotify.com/v1/tracks/5axqYa8jADY4...   
4  https://api.spotify.com/v1/tracks/0fqjy5gVNVmQ...   
5  https://api.spotify.com/v1/tracks/68vdi4VhdQ3J...   
6  https://api.spotify.com/v1/tracks/4ZmAMOU0bcmr...   
7  https://api.spotify.com/v1/tracks/3mY5W2fJF9wK...   
8  https://api.spotify.com/v1/tracks/0wl2rHiiQRHA...   
9  https://api.spotify.com/v1/tracks/1lIz3Wn8b1lB...   

                                        analys

## Clean Up

In [6]:
con.close()